In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [1]:
import numpy as np
import pandas as pd
import os
import gzip
import tensorflow as tf
from tensorflow import keras
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt

In [4]:
# 从内存中构建数据及
# 参数：列表、numpy数组、字典、元组等等
dataset = tf.data.Dataset.from_tensor_slices(np.arange(10))
print(dataset)

<TensorSliceDataset shapes: (), types: tf.int32>


In [6]:
# 对dataset遍历
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [7]:
# 1、repeat -- epoch
# 2、get batch
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [8]:
# interleave: 对现有dataset中每个元素处理，合并处理后产生的结果
# 常见的case：多个文件读取，合并成一个大的文件（数据集）

dataset2 = dataset.interleave(
    # map_fn：做什么变化
    # cycle_length: 并行程度
    # block_length：每次取多少元素处理
    lambda v: tf.data.Dataset.from_tensor_slices(v), # 取出v并封装成新的数据集
    cycle_length=5,
    block_length=5   
    # dataset中每个张量取前5个 取完 8 9不够了 就第一个tensor剩下的5 6，再不够第二个tensor的2
    # 达到均匀混合的效果
)
for item in dataset2:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype

In [10]:
# x = examples y = label 第一个维度需要相等
x = np.array([[1, 2], [3, 4], [5, 6]])
y = np.array(['cat', 'dog', 'fox'])
dataset3 = tf.data.Dataset.from_tensor_slices((x, y))
print(dataset3)

for item_x, item_y in dataset3:
    print(item_x, item_y)
    
for item_x, item_y in dataset3:
    print(item_x.numpy(), item_y.numpy())

<TensorSliceDataset shapes: ((2,), ()), types: (tf.int32, tf.string)>
tf.Tensor([1 2], shape=(2,), dtype=int32) tf.Tensor(b'cat', shape=(), dtype=string)
tf.Tensor([3 4], shape=(2,), dtype=int32) tf.Tensor(b'dog', shape=(), dtype=string)
tf.Tensor([5 6], shape=(2,), dtype=int32) tf.Tensor(b'fox', shape=(), dtype=string)
[1 2] b'cat'
[3 4] b'dog'
[5 6] b'fox'


In [16]:
dataset4 = tf.data.Dataset.from_tensor_slices({"feature":x, "label": y})
print(dataset4)
for item in dataset4:
    print(item)
    
for item in dataset4:
    print(item["feature"].numpy(), item["label"].numpy())

<TensorSliceDataset shapes: {feature: (2,), label: ()}, types: {feature: tf.int32, label: tf.string}>
{'feature': <tf.Tensor: id=219, shape=(2,), dtype=int32, numpy=array([1, 2])>, 'label': <tf.Tensor: id=220, shape=(), dtype=string, numpy=b'cat'>}
{'feature': <tf.Tensor: id=221, shape=(2,), dtype=int32, numpy=array([3, 4])>, 'label': <tf.Tensor: id=222, shape=(), dtype=string, numpy=b'dog'>}
{'feature': <tf.Tensor: id=223, shape=(2,), dtype=int32, numpy=array([5, 6])>, 'label': <tf.Tensor: id=224, shape=(), dtype=string, numpy=b'fox'>}
[1 2] b'cat'
[3 4] b'dog'
[5 6] b'fox'
